In [1]:

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import torch


from bertviz.pytorch_transformers_attn import BertModel, BertTokenizer
from bertviz.head_view_vilbert import show
from vilbert.datasets import ConceptCapLoaderTrain, ConceptCapLoaderVal
from bertviz.pytorch_transformers_attn.vilbert import VILBertForVLTasks, BertConfig, BertForMultiModalPreTraining

import numpy as np
import matplotlib.pyplot as plt
import PIL
%matplotlib inline  

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min',
    jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
# Load pre-trained model (weights)
from types import SimpleNamespace

root_path = '/coc/pskynet2/jlu347/multi-modal-bert/'

args = SimpleNamespace(validation_file=root_path + "data/conceptual_caption/validation",
                       from_pretrained= root_path + "save/bert_base_6_layer_6_connect_freeze_0/pytorch_model_8.bin",
                       bert_model="bert-base-uncased",
                       output_dir=root_path + 'save',
                       config_file=root_path + "config/bert_base_6layer_6conect.json",
                       max_seq_length=36,
                       train_batch_size=1,
                       do_lower_case=True,
                       predict_feature=False,
                       seed=42,
                       num_workers=0,
                       baseline=False,
                       img_weight=1,
                       distributed=False,
                      )
config = BertConfig.from_json_file(args.config_file)
if args.predict_feature:
    config.v_target_size = 2048
    config.predict_feature = True
else:
    config.v_target_size = 1601
    config.predict_feature = False

if args.from_pretrained:
    model = BertForMultiModalPreTraining.from_pretrained(args.from_pretrained, config)
else:
    model = BertForMultiModalPreTraining(config)
    
model.eval()
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

validation_dataset = ConceptCapLoaderVal(
    args.validation_file,
    tokenizer,
    seq_len=args.max_seq_length,
    batch_size=args.train_batch_size,
    predict_feature=args.predict_feature,
    num_workers=0,
    distributed=args.distributed,
    visualization=True
)

07/28/2019 21:58:37 - INFO - bertviz.pytorch_transformers_attn.vilbert -   loading archive file /coc/pskynet2/jlu347/multi-modal-bert/save/bert_base_6_layer_6_connect_freeze_0/pytorch_model_8.bin
07/28/2019 21:58:37 - INFO - bertviz.pytorch_transformers_attn.vilbert -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "bi_attention_type": 1,
  "bi_hidden_size": 1024,
  "bi_intermediate_size": 1024,
  "bi_num_attention_heads": 8,
  "fast_mode": false,
  "fixed_t_layer": 0,
  "fixed_v_layer": 0,
  "fusion_method": "mul",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "in_batch_pairs": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "intra_gate": false,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooling_method": "mul",
  "predict_feature": false,
  "t_biattention_id": [
    6,
    7,
    8,
    9,
    10,
    11
  ],
  "type_vocab_size": 2,
  "v_attention_probs_dropout_prob": 0.1,
  

model's option for predict_feature is  False


07/28/2019 21:58:44 - INFO - bertviz.pytorch_transformers_attn.vilbert -   Weights from pretrained model not used in BertForMultiModalPreTraining: ['bert.encoder.c_layer.0.biattention.scale.weight', 'bert.encoder.c_layer.1.biattention.scale.weight', 'bert.encoder.c_layer.2.biattention.scale.weight', 'bert.encoder.c_layer.3.biattention.scale.weight', 'bert.encoder.c_layer.4.biattention.scale.weight', 'bert.encoder.c_layer.5.biattention.scale.weight']
07/28/2019 21:58:50 - INFO - bertviz.pytorch_transformers_attn.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /nethome/jlu347/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Loading from /coc/pskynet2/jlu347/multi-modal-bert/data/conceptual_caption/validation_feat_all.lmdb
[0728 21:58:50 @format.py:92] Found 6130 entries in /coc/pskynet2/jlu347/multi-modal-bert/data/conceptual_caption/validation_feat_all.lmdb


In [4]:
def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))

In [ ]:
# sample a batch
import random
burn_in = int(random.random() * 5000)
print(burn_in)
for step, batch in enumerate(validation_dataset):
    image_id = batch[-1][0]

    batch = tuple(t.cuda() for t in list(batch)[:-1])
    input_ids, input_mask, segment_ids, lm_label_ids, is_next, image_feat, image_loc, image_target, image_label, image_mask = (
        batch
    )
    
    if step > burn_in and is_next[0] == 0:
        break

# image_id = int(image_ids[0].item())
# show the image.

image_path = '/srv/datasets/conceptual_caption/validation/%s' %image_id
img = PIL.Image.open(image_path).convert('RGB')
img = torch.tensor(np.array(img))
plt.axis('off')
plt.imshow(img)
plt.show()
# show the caption.

sents = input_ids.cpu().numpy().tolist()[0]
sents = tokenizer.convert_ids_to_tokens(sents)
sents = [sent for sent in sents if sent != "[PAD]"]
print(" ".join(sents))

In [ ]:
# show the patch for each image index
num = image_mask.sum().item()
image_label_list = image_label[0].tolist()
image_label_list.insert(0,-1)
height, width, _ = img.shape
print("image height: %d, width %d" %(height, width))

examples_per_row = 5
nrows, ncols = int(num / examples_per_row)+1, examples_per_row 
figsize = [ncols*3, ncols*5]     # figure size, inches
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

for i, axi in enumerate(ax.flat):
    if i < num:
        box = image_loc[0][i][:4].tolist()
        y1 = int(box[1] * height)
        y2 = int(box[3] * height)
        x1 = int(box[0] * width)
        x2 = int(box[2] * width)
#         print([y1,x1,y2,x2])
        patch = img[y1:y2,x1:x2]
        axi.imshow(patch)
        axi.axis('off')
        if image_label_list[i] == 1:
            axi.set_title(str(i) + '(masked)')
        else:
            axi.set_title(str(i))
            
plt.axis('off')
plt.tight_layout(True)
plt.show()  

In [ ]:
from bertviz.model_view_vilbert import show
show(model, tokenizer, batch)